In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [18]:
from scipy.interpolate import RectBivariateSpline
from skimage.transform import resize

In [19]:
patches_rootdir = '/home/yuncong/CSHL_data_patches'

In [20]:
# scoremaps_rootdir = '/home/yuncong/CSHL_scoremaps_lossless_inceptionModel'
scoremaps_rootdir = '/home/yuncong/CSHL_scoremaps_lossless'
if not os.path.exists(scoremaps_rootdir):
    os.makedirs(scoremaps_rootdir)
    
# predictions_rootdir = '/home/yuncong/CSHL_patch_predictions_inceptionModel'
predictions_rootdir = '/home/yuncong/CSHL_patch_predictions'
if not os.path.exists(predictions_rootdir):
    os.makedirs(predictions_rootdir)

In [21]:
# for stack in ['MD585']:
# for stack in ['MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590']:
# for stack in ['MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
for stack in ['MD594']:

    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    for sec in range(196, last_detect_sec+1):
#     for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in [142]:

        if sec not in indices_allROIs_allSections.columns:
            continue

        print sec

        indices_roi = indices_allROIs_allSections[sec]['roi1']

        predictions_dir = os.path.join(predictions_rootdir, stack, '%04d'%sec)
        if not os.path.exists(predictions_dir):
            os.makedirs(predictions_dir)

        scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
        if not os.path.exists(scoremaps_dir):
            os.makedirs(scoremaps_dir)

        ## define grid, generate patches

        t = time.time()

        dm = DataManager(stack=stack, section=sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]
        sample_locations_roi = sample_locations[indices_roi]
        
        ## interpolate

        interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
        interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
        interpolation_w = interpolation_xmax - interpolation_xmin + 1
        interpolation_h = interpolation_ymax - interpolation_ymin + 1

        ##### sample_locations_roi + scores to dense_score_map #####

        shrink_factor = 4 # do interpolation on a smaller grid, then resize to original dimension

        sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
        sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])

        n_sample_x = sample_locations_unique_xs.size
        n_sample_y = sample_locations_unique_ys.size

        index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
        index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
        sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])

        sample_locations_interpolatedArea_ys_matrix, \
        sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                        interpolation_ymax/shrink_factor), 
                                                                  range(interpolation_xmin/shrink_factor, 
                                                                        interpolation_xmax/shrink_factor), 
                                                                  indexing='ij')


        sys.stderr.write('preprocess: %.2f seconds\n' % (time.time() - t))
        
        dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
        probs = np.load(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset})

        
        def generate_score_map(l):

#             label = labels[(l+1)%len(labels)]
            label = labels[l]

            if label == 'BackG':
                return

            score_matrix = np.zeros((n_sample_x, n_sample_y))
            score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs[:,l]

            
            spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                         sample_locations_unique_ys/shrink_factor, 
                                         score_matrix, 
                                         bbox=[interpolation_xmin/shrink_factor, 
                                               interpolation_xmax/shrink_factor, 
                                               interpolation_ymin/shrink_factor, 
                                               interpolation_ymax/shrink_factor])

            dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, 
                                        sample_locations_interpolatedArea_ys_matrix)

            dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w))

            dense_score_map[dense_score_map < 1e-2] = 0
            
            if np.count_nonzero(dense_score_map[::8, ::8] > .1) < 1e3:
                sys.stderr.write('No %s is detected on section %d\n' % (label, sec))
                return
            
#             dense_score_map_lossless = np.pad(dense_score_map, ((interpolation_ymin, h-interpolation_ymax-1), 
#                                                   (interpolation_xmin, w-interpolation_xmax-1)),
#                                               mode='constant', constant_values=0)
            
#             t = time.time()
            bp.pack_ndarray_file(dense_score_map.astype(np.float32), 
                                   os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s.bp' % \
                                                {'dataset': dataset, 'label': label}))
    
#             filters = Filters(complevel=9, complib='blosc')
#             with open_file( os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s.hdf' % \
#                                                             {'dataset': dataset, 'label': label}), mode="w") as f:
#                 _ = f.create_carray('/', 'data', Atom.from_dtype(dense_score_map_lossless.dtype), filters=filters, obj=dense_score_map_lossless)
    
            np.savetxt(os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': label}),
                   np.array((interpolation_xmin, interpolation_xmax, interpolation_ymin, interpolation_ymax))[None], 
                   fmt='%d')
    
#             sys.stderr.write('write: %.2f seconds\n' % (time.time() - t))
        
#             return dense_score_map_lossless

        
        t = time.time()

        n_labels = probs.shape[1]

        _ = Parallel(n_jobs=12)(delayed(generate_score_map)(l) for l in range(n_labels))

        sys.stderr.write('interpolate: %.2f seconds\n' % (time.time() - t))

196


preprocess: 4.19 seconds
interpolate: 20.02 seconds
preprocess: 3.96 seconds


197


interpolate: 56.51 seconds
preprocess: 3.94 seconds
No SuVe is detected on section 197
No Gr is detected on section 197
No 6N is detected on section 197


198


...

interpolate: 26.13 seconds
preprocess: 3.64 seconds
No Amb is detected on section 303
No Sol is detected on section 303


304


interpolate: 16.31 seconds
preprocess: 3.63 seconds
No 12N is detected on section 304


305


interpolate: 17.01 seconds
No 12N is detected on section 305
No Tz is detected on section 305
